In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
deliveries = pd.read_csv('../input/ipl-data-upto-2020-and-schedule-2021/deliveries_2008-2020 (1).csv')
matches = pd.read_csv('../input/ipl-data-upto-2020-and-schedule-2021/matches_2008-2020 (1).csv')
merged = pd.read_csv('../input/ipl-data-upto-2020-and-schedule-2021/merged_data.csv')

In [ ]:
deliveries[deliveries['batsman']=='V Kohli'].batsman_runs.sum()

In [ ]:
deliveries[deliveries['batsman']=='V Kohli'].batsman_runs.value_counts()

In [ ]:
matches = pd.read_csv('../input/ipl-2020-ball-by-ball-data/Qualifiers IPL 2008-2019.csv')
deliveries = pd.read_csv('../input/ipl-2020-ball-by-ball-data/Deliveries IPL 2020.csv')

In [ ]:
deliveries[(deliveries.batsman=='V Kohli')&(deliveries.is_super_over==0)]

In [ ]:
# score predictor
"""
- over
- ball
- batsman_strike rate_in the match   # Done
- non-strike batsman strike rate in the match# Done 
- batsman_score # Done
- non_strike batsman_score # Done
- batsman_boundary% # Done
- non_strike batsman_boundary% #Done
- wickets_fall # Done
- venue_average score_last 5 match
- team average score batting first
- team average score batting first against bowling team
- team average score batting first at venue
- 
"""

In [ ]:
# First inning deleveries
deliveries = df[df['inning']==1]

In [ ]:
matchId = 1
over = 2
ball = 3
batsman = 'DA Warner'
mask1= deliveries['match_id']==matchId
mask2 = deliveries['batsman']==batsman
mask3 = deliveries['over'] < over
mask4 = (deliveries['ball']<=ball) & (deliveries['over']==over)
mask5 = deliveries['wide_runs']==0
temp = deliveries[mask1 & mask2 & (mask3|mask4) & mask5]
strike_rate = temp.batsman_runs.sum()/temp.count()['ball'] * 100
strike_rate

In [ ]:
def batsman_strike_rate_in_the_match(batsman, matchId, over, ball):
#     mask = deliveries['inning']==1
    mask1= deliveries['match_id']==matchId
    mask2 = deliveries['batsman']==batsman
    mask3 = deliveries['over'] < over
    mask4 = (deliveries['ball']<=ball) & (deliveries['over']==over)
    mask5 = deliveries['wide_runs']==0
    temp = deliveries[mask1 & mask2 & (mask3|mask4) & mask5]
    strike_rate = temp.batsman_runs.sum()/temp.count()['ball'] * 100
    return strike_rate

In [ ]:
batsman_strike_rate_in_the_match(batsman, matchId, over, ball)

In [ ]:
def batsman_boundary_percent_in_the_match(batsman, matchId, over, ball):
#     mask = deliveries['inning']==1
    mask1= deliveries['match_id']==matchId
    mask2 = deliveries['batsman']==batsman
    mask3 = deliveries['over'] < over
    mask4 = (deliveries['ball']<=ball) & (deliveries['over']==over)
    mask5 = deliveries['wide_runs']==0
    temp = deliveries[mask1 & mask2 & (mask3|mask4) & mask5]
    total_runs = temp.batsman_runs.sum()
    boundary_runs = temp[(temp['batsman_runs']==4) | (temp['batsman_runs']==6)]['batsman_runs'].sum()
    return boundary_runs/total_runs*100

In [ ]:
batsman_boundary_percent_in_the_match(batsman, matchId, over, ball)

In [ ]:
def batsman_score_in_the_match(batsman, matchId, over, ball):
#     mask = deliveries['inning']==1
    mask1= deliveries['match_id']==matchId
    mask2 = deliveries['batsman']==batsman
    mask3 = deliveries['over'] < over
    mask4 = (deliveries['ball']<=ball) & (deliveries['over']==over)
    mask5 = deliveries['wide_runs']==0
    temp = deliveries[mask1 & mask2 & (mask3|mask4) & mask5]
    batsman_runs = temp.batsman_runs.sum()
    return batsman_runs

In [ ]:
batsman_score_in_the_match(batsman, matchId, over, ball)

In [ ]:
def wickets_fallen(matchId, over, ball):
#     mask = deliveries['inning']==1
    mask1= deliveries['match_id']==matchId
#     mask2 = deliveries['batsman']==batsman
    mask3 = deliveries['over'] < over
    mask4 = (deliveries['ball']<=ball) & (deliveries['over']==over)
    mask5 = deliveries.dismissal_kind.isin(['caught', 'bowled', 'run out', 'lbw', 'caught and bowled', 'stumped', 'retired hurt', 'hit wicket', 'obstructing the field'])
    temp = deliveries[mask1 & (mask3|mask4) & mask5]
    
    return temp.count()['dismissal_kind']

In [ ]:
wickets_fallen(matchId, 19, 4)

In [ ]:
def venue_avg_score(matchId):
    match_ids = matches[matches['venue']==matches[matches['id']==matchId]['venue'].values[0]]['id'].values
    temp = deliveries[deliveries['match_id'].isin(match_ids)].groupby('match_id')['total_runs'].sum()
    return sum(temp)/len(temp)

In [ ]:
venue_avg_score(matchId)

In [ ]:
def team_avg_score(team):
#     deliveries = deliveries[deliveries['inning']==1]
    temp = deliveries[(deliveries['batting_team']==team)].groupby('match_id')['total_runs']
    return sum(temp.sum())/len(temp)

In [ ]:
team_avg_score('Sunrisers Hyderabad')

In [ ]:
def team_avg_score_venue(team, matchId):
#     deliveries = deliveries[deliveries['inning']==1]
    match_ids = matches[matches['venue']==matches[matches['id']==matchId]['venue'].values[0]]['id'].values
    temp = deliveries[(deliveries['match_id'].isin(match_ids))&(deliveries['batting_team']==team)]
    temp = temp.groupby('match_id')['total_runs']
    return sum(temp.sum())/len(temp)

In [ ]:
team_avg_score_venue('Sunrisers Hyderabad', 1)

In [ ]:
def team_avg_score_against_bowling_team(bat_team, bowl_team):
#     deliveries = deliveries[deliveries['inning']==1]
    temp = deliveries[(deliveries['bowling_team']==bowl_team)&(deliveries['batting_team']==bat_team)]
    temp = temp.groupby('match_id')['total_runs']
    return sum(temp.sum())/len(temp)

In [ ]:
team_avg_score_against_bowling_team('Sunrisers Hyderabad','Royal Challengers Bangalore')

In [ ]:
match_ids = df.match_id.unique()

In [ ]:
temp = df.groupby('match_id')

In [ ]:
df.dismissal_kind.unique()

In [ ]:
dismissal_kind = ['caught', 'bowled', 'lbw', 'caught and bowled', 'run out', 'stumped', 'hit wicket', 'retired hurt']

In [ ]:
data.iloc[1]

In [ ]:
wickets_list=[]
runs_list=[]
ids = []
for i in match_ids:
    data = temp.get_group(i)
    wickets = 0
    runs=0
    for j in range(data.shape[0]):
        if data.iloc[j]['dismissal_kind'] in dismissal_kind:
            wickets=wickets+1  
        runs = runs + data.iloc[j]['total_runs']
        runs_list.append(runs)
        wickets_list.append(wickets)
        ids.append(i)
    

In [ ]:
len(ids)

In [ ]:
dfx = pd.DataFrame(columns=['match_id','wickets_fallen'])

In [ ]:
dfx['match_id'] = ids

In [ ]:
dfx['wickets_fallen'] = array

In [ ]:
df.shape

In [ ]:
dfx.shape

In [ ]:
df_new = df.join(dfx, on='match_id', lsuffix='new')

In [ ]:
df_new.shape

In [ ]:
df_new.wickets_fallen=df_new.wickets_fallen.convert_dtypes('int')

In [ ]:
df_new.wickets_fallen=df_new.wickets_fallen.convert_dtypes('int')

In [ ]:
df_new.info()

In [ ]:
# hat-trick

In [ ]:
merged.columns

In [ ]:
merged.sort_values(['match_id','inning','over','ball'], inplace=True)

In [ ]:
merged

In [ ]:
filtered = merged[(merged.wide_runs==0)&(merged.noball_runs==0)]

In [ ]:
filtered[filtered.bowler=='A Mishra'].head(2)[['bowler']]

In [ ]:
xyz = (filtered[(filtered['bowler']=='A Mishra')&(filtered.player_dismissed)].groupby('match_id').count()>=3)['id']

In [ ]:
xyz[xyz].index

In [ ]:
xyz[(filtered[(filtered['bowler']=='A Mishra')&(filtered.player_dismissed)].groupby('match_id').count()>=3)['id']].index

In [ ]:
xyz = filtered[(filtered['bowler']=='A Mishra')&(filtered.player_dismissed)].groupby('match_id')
abc = (filtered[(filtered['bowler']=='A Mishra')&(filtered.player_dismissed)].groupby('match_id').count()>=3)['id']
ids = abc[abc]

In [ ]:
# Is must for no. Of hatricks function 
is_bowler_wicket = [1 if x in (['caught', 'bowled','lbw', 'caught and bowled','stumped', 'hit wicket',]) else 0 for x in merged.dismissal_kind]
merged['is_bowler_wicket']=is_bowler_wicket

In [ ]:
# Function for finding hattricks in IPL_ by bowler

def noOfHattricks(bowler,year=None):
    
    #merged  is deliveries plus matches sorted on matchid inning over ball
    years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
    if year:
        years=[year]
        
    merged.sort_values(['match_id','inning','over','ball'], inplace=True)
    filtered = merged[(merged['bowler']==bowler)&(merged['season'].isin(years))]
    
    # For extracting ids where bowler has taken 3 or more wickets
    pqr = filtered[(filtered.dismissal_kind.isin(['caught', 'bowled','lbw', 'caught and bowled','stumped', 'hit wicket',]))].groupby('match_id')
    abc = (pqr.count()>=3)['id']
    ids = abc[abc].index
    
    # For checking wetheer those are hattric or not-- we need all balls data
#     xyz = filtered[(filtered.dismissal_kind.isin([np.nan,'caught', 'bowled','lbw', 'caught and bowled','stumped', 'hit wicket',]))].groupby('match_id')
    xyz= filtered.groupby('match_id')
    
    count=0
    match_id = []
    for j in ids:
        df = xyz.get_group(j)
        for i in range(df.shape[0]-2):
            if (df.iloc[i:i+3]['is_bowler_wicket'].sum()==3):
                count+=1
                match_id.append(j)
                print('Bowler:- ',bowler,count)
                print('0-Over: ',df.iloc[i]['over'],' ball: ',df.iloc[i]['ball'],)
#             if ((df.iloc[i:i+3]['is_bowler_wicket'].sum()==3)&(df.iloc[i]['over']==df.iloc[i+1]['over']==df.iloc[i+2]['over'])):
#                 if (df.iloc[i+2]['ball']-1==df.iloc[i+1]['ball']==df.iloc[i]['ball']+1):
#                     count+=1
#                     match_id.append(j)
#                     print('Bowler:- ',bowler,count)
#                     print('1--Over: ',df.iloc[i]['over'],' ball: ',df.iloc[i]['ball'],)
#             elif ((df.iloc[i:i+2]['is_bowler_wicket'].sum()==2)&(((df.iloc[i]['ball']==5)&(df.iloc[i+1]['ball']==6))or((df.iloc[i]['ball']==6)&(df.iloc[i+1]['ball']==7))or((df.iloc[i]['ball']==7)&(df.iloc[i+1]['ball']==8)))):
#                 if ((df.iloc[i+2]['is_bowler_wicket'])&(df.iloc[i+2]['ball']==1)):
#                     count+=1
#                     match_id.append(j)
#                     print('Bowler:- ',bowler,count)
#                     print('2--Over: ',df.iloc[i]['over'],' ball: ',df.iloc[i]['ball'],)
#             elif ((df.iloc[i]['is_bowler_wicket'])&((df.iloc[i]['ball']==6)|(df.iloc[i]['ball']==7)|(df.iloc[i]['ball']==8))):
#                 if ((df.iloc[i+1:i+3]['is_bowler_wicket'].sum()==2)&(df.iloc[i+1]['ball']==1)&(df.iloc[i+2]['ball']==2)):
#                     count+=1
#                     match_id.append(j)
#                     print('Bowler:- ',bowler,count)
#                     print('3--Over: ',df.iloc[i]['over'],' ball: ',df.iloc[i]['ball'],)
#     print(count,match_id)
    return count,match_id, xyz

In [ ]:
# Function for finding hattricks in IPL_ by bowler-- must include is_bowler_wicket in merged dataframe
def noOfHattricks(bowler,year=None):
    #merged  is deliveries plus matches sorted on matchid inning over ball
    years=[2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
    if year:
        years=[year]
        
    merged.sort_values(['match_id','inning','over','ball'], inplace=True)
    filtered = merged[(merged['bowler']==bowler)&(merged['season'].isin(years))]
    
    # For extracting ids where bowler has taken 3 or more wickets
    xyz = filtered.groupby('match_id')
    abc = (xyz.sum()['is_bowler_wicket']>=3)
    ids = abc[abc].index
    
    count=0
    match_id = []
    for j in ids:
        df = xyz.get_group(j)
        for i in range(df.shape[0]-2):
            if (df.iloc[i:i+3]['is_bowler_wicket'].sum()==3):
                count+=1
                match_id.append(j)
    if count:
        print('Bowler:- ',bowler,count)
    return count,match_id, xyz

In [ ]:
c,b,xyz = noOfHattricks('K Rabada')

In [ ]:
asd = merged.groupby(['match_id','bowler']).sum()
potential_bowlers = asd[asd.is_bowler_wicket>=3].reset_index()['bowler'].unique()
len(potential_bowlers)

In [ ]:
no_hat=0
hattricks=[]
for b in potential_bowlers:
    p,match_id,_=noOfHattricks(b)
    no_hat+=p
    if p:
        hattricks.append([b,p,match_id])
no_hat

In [ ]:
len(hattricks)